# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [6]:
!more publications.tsv

pub_date        title   Citation        URL     paper_filename
2023    Evolution and impact of high content imaging    "Gregory Way, Heba Sailem, Steven Shave, Richard Kasprowicz, Neil Carragher, ""Evolution and impact of high content imaging."" Slas Discovery, 2023."   https://slas-discovery.org/article/S2472-5552(23)00066-7/fulltext       2023_Evolution_and_impact_of_HCI.pdf
2023    Phenonaut: multiomics data integration for phenotypic space exploration "Steven Shave, John Dawson, Abdullah Athar, Cuong Nguyen, Richard Kasprowicz, Neil Carragher, ""Phenonaut: multiomics data integration for phenotypic space exploration."" Bioinformatics, 2023."       https://academic.oup.com/bioinformatics/article/39/4/btad143/7082955    2023_Phenonaut.pdf
2022    Chemical interrogation of nuclear size identifies compounds with cancer cell line-specific effects on migration and invasion    "Sylvain Tollis, Andrea Rizzotto, Nhan Pham, Sonja Koivukoski, Aishwarya Sivakumar, Steven Shave, Jan Wildenhain,

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [27]:
import pandas as pd
publications_df = pd.read_csv("publications.tsv", sep="\t", header=0,  encoding = "ISO-8859-1")
publications_df


,pub_date,title,citation,url,doi,pdf_filename,code
0,2023,From Intuition to AI: Evolution of Small Molec...,"Miles McGibbon, Steven Shave, Jie Dong, Yumiao...",NaN,NaN,NaN,NaN
1,2023,OpenFEPOPS: A Python implementation of the FEP...,"Yan-Kai Chen, Douglas Houston, Manfred Auer, a...",NaN,NaN,NaN,NaN
2,2023,Evolution and impact of high content imaging,"Gregory Way, Heba Sailem, Steven Shave, Richar...",https://slas-discovery.org/article/S2472-5552(...,https://doi.org/10.1016/j.slasd.2023.08.009,2023_Evolution_and_impact_of_HCI.pdf,NaN
3,2023,Phenonaut: multiomics data integration for phe...,"Steven Shave, John Dawson, Abdullah Athar, Cuo...",https://academic.oup.com/bioinformatics/articl...,https://doi.org/10.1093/bioinformatics/btad143,2023_Phenonaut.pdf,https://github.com/CarragherLab/phenonaut
4,2022,Chemical interrogation of nuclear size identif...,"Sylvain Tollis, Andrea Rizzotto, Nhan Pham, So...",https://pubs.acs.org/doi/10.1021/acschembio.2c...,https://doi.org/10.1021/acschembio.2c00004,2022_Chemical_Interrogation_of_Nuclear_Size.pdf,NaN
5,2022,CLAffinity: A Software Tool for Identification...,"Steven Shave, Nhan Pham, Manfred Auer, ""CLAffi...",https://pubs.acs.org/doi/10.1021/acs.jcim.2c00285,https://doi.org/10.1021/acs.jcim.2c00285,2022_CLAffinity.pdf,https://github.com/stevenshave/competition-lab...
6,2021,SimilarityLab: molecular similarity for SAR ex...,"Steven Shave, Manfred Auer, ""SimilarityLab: mo...",https://www.mdpi.com/2227-9717/9/9/1520,https://doi.org/10.3390/pr9091520,2021_SimilarityLab.pdf,https://github.com/stevenshave/similaritylab
7,2021,Mrlogp: transfer learning enables accurate log...,"Yan-Kai Chen, Steven Shave, Manfred Auer, ""Mrl...",https://www.mdpi.com/2227-9717/9/11/2029,https://doi.org/10.3390/pr9112029,2021_MRlogP.pdf,https://github.com/JustinYKC/MRlogP
8,2021,"PyBindingCurve, simulation, and curve fitting ...","Steven Shave, Yan-Kai Chen, Nhan Pham, Manfred...",https://pubs.acs.org/doi/10.1021/acs.jcim.1c00216,https://doi.org/10.1021/acs.jcim.1c00216,2021_PyBindingCurve.pdf,https://github.com/stevenshave/pybindingcurve
9,2021,A two-step resin based approach to reveal surv...,"Ambrose, Andrew J., et al. ""A two-step resin b...",https://pubs.rsc.org/en/content/articlehtml/20...,https://doi.org/10.1039/D0CB00122H,2021_A_twostep_resin_approach_for_survivin_sel...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [28]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text:str):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

def bold_sumbitted_accepted(text:str):
    words=['Submitted: ', 'Accepted: ']
    for word in words:
        if word in text:
            return text.replace(word, f"<b>{word}</b>")
    return text

def underline_sshave(text:str):
    sshave=['Steven Shave', 'S. Shave', 'Shave, Steven', 'Shave, S.', 'Shave, S. R.']
    for name in sshave:
        if name in text:
            return text.replace(name, f"<u>{name}</u>")
    print("Warning, none of", sshave, "found in text: ", text)
    return text

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [29]:
import os

with open("../_pages/publications_list.md", 'w') as f:
    for idx, row in publications_df.iterrows():
        if pd.notna(row.doi) and len(row.doi)>5:
            md=f'### <a href="{row.doi}">{row.title}</a>\n\n'
        else:
            md=f"### {row.title}\n\n"
        citation=underline_sshave(row.citation)
        
        
        
        
        
        md+=f"{citation}\n\n"
        if pd.notna(row.doi) and len(row.doi)>5:
            md+=f'<a href="{row.doi}"><i class="fas fa-fw fa-link"></i></a>\n'
        if pd.notna(row.url) and len(row.url)>5:
            md+=f'<a href="{row.url}"><i class="fas fa-fw fa-newspaper"></i></a>\n'
        if pd.notna(row.pdf_filename) and len(row.pdf_filename)>5:
            md+=f'<a href="{{{{site.url}}}}/files/publications/{row.pdf_filename}"><i class="fas fa-fw fa-file-pdf fa-xl" aria-hidden="true"></i></a>\n'
        if pd.notna(row.code) and len(row.code)>5:
            md+=f'<a href="{row.code}"><i class="fab fa-fw fa-github zoom"></i></a>\n'
        md+="\n"
        
        f.write(md)

Warning, none of ['Steven Shave', 'S. Shave', 'Shave, Steven', 'Shave, S.', 'Shave, S. R.'] found in text:  Ambrose, Andrew J., et al. "A two-step resin based approach to reveal survivin-selective fluorescent probes." RSC Chemical Biology 2.1 (2021): 181-186.
Warning, none of ['Steven Shave', 'S. Shave', 'Shave, Steven', 'Shave, S.', 'Shave, S. R.'] found in text:  Kennedy, Susan A., et al. "Extensive rewiring of the EGFR network in colorectal cancer cells expressing transforming levels of KRASG13D." Nature communications 11.1 (2020): 499.


These files are in the publications directory, one directory below where we're working from.

In [23]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).